In [1]:
import os
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import Tool, initialize_agent
from langchain.llms import OpenAI

In [2]:
llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

agent = initialize_agent(
    tools, llm, agent="zero-shot-react-description", verbose=False
)

In [11]:
# No tracing
agent.run(
    "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
)



> Entering new LLMMathChain chain...
36^2
Answer: 1296
> Finished LLMMathChain chain.


"Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."

In [12]:
# Stdout Tracing
os.environ["LANGCHAIN_TRACER"] = "stdout"

agent.run(
    "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
)

Starting chain trace with inputs: {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"}, serialized: {'name': 'AgentExecutor'}, extra: {}
Starting chain trace with inputs: {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?", 'agent_scratchpad': '', 'stop': ['\nObservation: ']}, serialized: {'name': 'LLMChain'}, extra: {}
Starting LLM trace with prompts: ["Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events\nCalculator: useful for when you need to answer questions about math\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action

"Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."

In [3]:
# Nested JSON tracing
os.environ["LANGCHAIN_TRACER"] = "nested_json"

agent.run(
    "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
)



> Entering new LLMMathChain chain...
36^2
Answer: 1296
> Finished LLMMathChain chain.
{
  "id": "fba89bd2-008f-48b5-8b11-33204bd82d20",
  "start_time": 1671893142.805878,
  "end_time": 1671893149.630651,
  "extra": {},
  "error": null,
  "execution_order": 1,
  "serialized": {
    "name": "AgentExecutor"
  },
  "inputs": {
    "input": "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
  },
  "outputs": {
    "output": "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."
  },
  "child_runs": [
    {
      "id": "b63203de-aebe-40b4-88f0-2684f9f35a51",
      "start_time": 1671893142.806007,
      "end_time": 1671893144.62983,
      "extra": {},
      "error": null,
      "execution_order": 2,
      "serialized": {
        "name": "LLMChain"
      },
      "inputs": {
        "input": "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
    

"Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."

In [1]:
chain_run = {
  "id": "27323fde-5701-4ed1-b2e9-01f8f55de25e",
  "start_time": 1671861636.461295,
  "end_time": 1671861643.92484,
  "extra": {},
  "error": None,
  "execution_order": 1,
  "serialized": {
    "name": "AgentExecutor"
  },
  "inputs": {
    "input": "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
  },
  "outputs": {
    "output": "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."
  },
  "child_runs": [
    {
      "id": "cffe3cc7-9a96-408c-8a22-e9837c47dd11",
      "start_time": 1671861636.469308,
      "end_time": 1671861638.553195,
      "extra": {},
      "error": None,
      "execution_order": 2,
      "serialized": {
        "name": "LLMChain"
      },
      "inputs": {
        "input": "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
        "agent_scratchpad": "",
        "stop": [
          "\nObservation: "
        ]
      },
      "outputs": {
        "text": " I need to find out who won the US Open men's tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: \"US Open men's tennis final 2019\""
      },
      "child_runs": [
        {
          "id": "83b5f76a-8955-407f-9b2e-2de186b1d01e",
          "start_time": 1671861636.470382,
          "end_time": 1671861638.55316,
          "extra": {},
          "error": None,
          "execution_order": 3,
          "serialized": {
            "name": "OpenAI"
          },
          "prompts": {
            "prompts": [
              "Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events\nCalculator: useful for when you need to answer questions about math\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: Who won the US Open men's tennis final in 2019? What is his age raised to the second power?\nThought:"
            ]
          },
          "response": [
            [
              " I need to find out who won the US Open men's tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: \"US Open men's tennis final 2019\""
            ]
          ]
        }
      ]
    },
    {
      "id": "1d7a9862-398a-469d-95d6-0e947b44b646",
      "start_time": 1671861638.553415,
      "end_time": 1671861638.637173,
      "extra": {},
      "error": None,
      "execution_order": 4,
      "serialized": {
        "name": "<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_en..."
      },
      "tool_input": "US Open men's tennis final 2019",
      "output": "Rafael Nadal won his 19th career Grand Slam title, and his fourth US Open crown, by surviving an all-time comback effort from Daniil ...",
      "action": "Search",
      "child_runs": []
    },
    {
      "id": "c2fd1278-e996-4eae-89f8-16d2baa41d02",
      "start_time": 1671861638.63722,
      "end_time": 1671861640.089096,
      "extra": {},
      "error": None,
      "execution_order": 5,
      "serialized": {
        "name": "LLMChain"
      },
      "inputs": {
        "input": "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
        "agent_scratchpad": " I need to find out who won the US Open men's tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: \"US Open men's tennis final 2019\"\nObservation: Rafael Nadal won his 19th career Grand Slam title, and his fourth US Open crown, by surviving an all-time comback effort from Daniil ...\nThought:",
        "stop": [
          "\nObservation: "
        ]
      },
      "outputs": {
        "text": " I need to find out Rafael Nadal's age\nAction: Search\nAction Input: \"Rafael Nadal age\""
      },
      "child_runs": [
        {
          "id": "94c8572e-d71f-48ec-bc6a-6b31c65f9030",
          "start_time": 1671861638.63733,
          "end_time": 1671861640.089062,
          "extra": {},
          "error": None,
          "execution_order": 6,
          "serialized": {
            "name": "OpenAI"
          },
          "prompts": {
            "prompts": [
              "Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events\nCalculator: useful for when you need to answer questions about math\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: Who won the US Open men's tennis final in 2019? What is his age raised to the second power?\nThought: I need to find out who won the US Open men's tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: \"US Open men's tennis final 2019\"\nObservation: Rafael Nadal won his 19th career Grand Slam title, and his fourth US Open crown, by surviving an all-time comback effort from Daniil ...\nThought:"
            ]
          },
          "response": [
            [
              " I need to find out Rafael Nadal's age\nAction: Search\nAction Input: \"Rafael Nadal age\""
            ]
          ]
        }
      ]
    },
    {
      "id": "34d234ef-6aff-45d7-a37b-35abb07224a0",
      "start_time": 1671861640.089209,
      "end_time": 1671861640.173572,
      "extra": {},
      "error": None,
      "execution_order": 7,
      "serialized": {
        "name": "<bound method SerpAPIWrapper.run of SerpAPIWrapper(search_en..."
      },
      "tool_input": "Rafael Nadal age",
      "output": "36 years",
      "action": "Search",
      "child_runs": []
    },
    {
      "id": "1a60c55f-8413-4c2c-b600-f7f787572e86",
      "start_time": 1671861640.173627,
      "end_time": 1671861641.522722,
      "extra": {},
      "error": None,
      "execution_order": 8,
      "serialized": {
        "name": "LLMChain"
      },
      "inputs": {
        "input": "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
        "agent_scratchpad": " I need to find out who won the US Open men's tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: \"US Open men's tennis final 2019\"\nObservation: Rafael Nadal won his 19th career Grand Slam title, and his fourth US Open crown, by surviving an all-time comback effort from Daniil ...\nThought: I need to find out Rafael Nadal's age\nAction: Search\nAction Input: \"Rafael Nadal age\"\nObservation: 36 years\nThought:",
        "stop": [
          "\nObservation: "
        ]
      },
      "outputs": {
        "text": " I now know the age and can calculate it raised to the second power\nAction: Calculator\nAction Input: 36^2"
      },
      "child_runs": [
        {
          "id": "f7ee83cd-9e0e-437c-88c7-7a1312b84aba",
          "start_time": 1671861640.173752,
          "end_time": 1671861641.522688,
          "extra": {},
          "error": None,
          "execution_order": 9,
          "serialized": {
            "name": "OpenAI"
          },
          "prompts": {
            "prompts": [
              "Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events\nCalculator: useful for when you need to answer questions about math\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: Who won the US Open men's tennis final in 2019? What is his age raised to the second power?\nThought: I need to find out who won the US Open men's tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: \"US Open men's tennis final 2019\"\nObservation: Rafael Nadal won his 19th career Grand Slam title, and his fourth US Open crown, by surviving an all-time comback effort from Daniil ...\nThought: I need to find out Rafael Nadal's age\nAction: Search\nAction Input: \"Rafael Nadal age\"\nObservation: 36 years\nThought:"
            ]
          },
          "response": [
            [
              " I now know the age and can calculate it raised to the second power\nAction: Calculator\nAction Input: 36^2"
            ]
          ]
        }
      ]
    },
    {
      "id": "558a5831-3a4f-4f96-9aeb-40b6eb72219a",
      "start_time": 1671861641.522874,
      "end_time": 1671861642.155347,
      "extra": {},
      "error": None,
      "execution_order": 10,
      "serialized": {
        "name": "<bound method Chain.run of LLMMathChain(memory=None, verbose..."
      },
      "tool_input": "36^2",
      "output": "Answer: 1296",
      "action": "Calculator",
      "child_runs": [
        {
          "id": "439fbd92-dd13-43f9-9951-1f84be3fd04e",
          "start_time": 1671861641.523141,
          "end_time": 1671861642.155305,
          "extra": {},
          "error": None,
          "execution_order": 11,
          "serialized": {
            "name": "LLMMathChain"
          },
          "inputs": {
            "question": "36^2"
          },
          "outputs": {
            "answer": "Answer: 1296"
          },
          "child_runs": [
            {
              "id": "45a614c1-2469-4a77-8318-88da74ca6f10",
              "start_time": 1671861641.523482,
              "end_time": 1671861642.155083,
              "extra": {},
              "error": None,
              "execution_order": 12,
              "serialized": {
                "name": "LLMChain"
              },
              "inputs": {
                "question": "36^2",
                "stop": [
                  "```output"
                ]
              },
              "outputs": {
                "text": "\nAnswer: 1296"
              },
              "child_runs": [
                {
                  "id": "5f03a32b-9c7a-4fe1-ad21-4d4e59842b6b",
                  "start_time": 1671861641.523634,
                  "end_time": 1671861642.155049,
                  "extra": {},
                  "error": None,
                  "execution_order": 13,
                  "serialized": {
                    "name": "OpenAI"
                  },
                  "prompts": {
                    "prompts": [
                      "You are GPT-3, and you can't do math.\n\nYou can do basic math, and your memorization abilities are impressive, but you can't do any complex calculations that a human could not do in their head. You also have an annoying tendency to just make up highly specific, but wrong, answers.\n\nSo we hooked you up to a Python 3 kernel, and now you can execute code. If anyone gives you a hard math problem, just use this format and we\u2019ll take care of the rest:\n\nQuestion: ${Question with hard calculation.}\n```python\n${Code that prints what you need to know}\n```\n```output\n${Output of your code}\n```\nAnswer: ${Answer}\n\nOtherwise, use this simpler format:\n\nQuestion: ${Question without hard calculation}\nAnswer: ${Answer}\n\nBegin.\n\nQuestion: What is 37593 * 67?\n\n```python\nprint(37593 * 67)\n```\n```output\n2518731\n```\nAnswer: 2518731\n\nQuestion: 36^2"
                    ]
                  },
                  "response": [
                    [
                      "\nAnswer: 1296"
                    ]
                  ]
                }
              ]
            }
          ]
        }
      ]
    },
    {
      "id": "bb599f4a-5632-4018-b6ed-2ecb4b22a7e5",
      "start_time": 1671861642.155405,
      "end_time": 1671861643.924784,
      "extra": {},
      "error": None,
      "execution_order": 14,
      "serialized": {
        "name": "LLMChain"
      },
      "inputs": {
        "input": "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
        "agent_scratchpad": " I need to find out who won the US Open men's tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: \"US Open men's tennis final 2019\"\nObservation: Rafael Nadal won his 19th career Grand Slam title, and his fourth US Open crown, by surviving an all-time comback effort from Daniil ...\nThought: I need to find out Rafael Nadal's age\nAction: Search\nAction Input: \"Rafael Nadal age\"\nObservation: 36 years\nThought: I now know the age and can calculate it raised to the second power\nAction: Calculator\nAction Input: 36^2\nObservation: Answer: 1296\nThought:",
        "stop": [
          "\nObservation: "
        ]
      },
      "outputs": {
        "text": " I now know the final answer\nFinal Answer: Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."
      },
      "child_runs": [
        {
          "id": "9c12a8ef-3217-4b8b-aa39-9c1a12aa5272",
          "start_time": 1671861642.155589,
          "end_time": 1671861643.924747,
          "extra": {},
          "error": None,
          "execution_order": 15,
          "serialized": {
            "name": "OpenAI"
          },
          "prompts": {
            "prompts": [
              "Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer questions about current events\nCalculator: useful for when you need to answer questions about math\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [Search, Calculator]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: Who won the US Open men's tennis final in 2019? What is his age raised to the second power?\nThought: I need to find out who won the US Open men's tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: \"US Open men's tennis final 2019\"\nObservation: Rafael Nadal won his 19th career Grand Slam title, and his fourth US Open crown, by surviving an all-time comback effort from Daniil ...\nThought: I need to find out Rafael Nadal's age\nAction: Search\nAction Input: \"Rafael Nadal age\"\nObservation: 36 years\nThought: I now know the age and can calculate it raised to the second power\nAction: Calculator\nAction Input: 36^2\nObservation: Answer: 1296\nThought:"
            ]
          },
          "response": [
            [
              " I now know the final answer\nFinal Answer: Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."
            ]
          ]
        }
      ]
    }
  ]
}

In [2]:
chain_run

{'id': '27323fde-5701-4ed1-b2e9-01f8f55de25e',
 'start_time': 1671861636.461295,
 'end_time': 1671861643.92484,
 'extra': {},
 'error': None,
 'execution_order': 1,
 'serialized': {'name': 'AgentExecutor'},
 'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
 'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
 'child_runs': [{'id': 'cffe3cc7-9a96-408c-8a22-e9837c47dd11',
   'start_time': 1671861636.469308,
   'end_time': 1671861638.553195,
   'extra': {},
   'error': None,
   'execution_order': 2,
   'serialized': {'name': 'LLMChain'},
   'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
    'agent_scratchpad': '',
    'stop': ['\nObservation: ']},
   'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raise

In [3]:
from langchain.tracing.base import ChainRun
cr = ChainRun.from_dict(chain_run)

In [4]:
cr.child_runs

[{'id': 'cffe3cc7-9a96-408c-8a22-e9837c47dd11',
  'start_time': 1671861636.469308,
  'end_time': 1671861638.553195,
  'extra': {},
  'error': None,
  'execution_order': 2,
  'serialized': {'name': 'LLMChain'},
  'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
   'agent_scratchpad': '',
   'stop': ['\nObservation: ']},
  'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: "US Open men\'s tennis final 2019"'},
  'child_runs': [{'id': '83b5f76a-8955-407f-9b2e-2de186b1d01e',
    'start_time': 1671861636.470382,
    'end_time': 1671861638.55316,
    'extra': {},
    'error': None,
    'execution_order': 3,
    'serialized': {'name': 'OpenAI'},
    'prompts': {'prompts': ["Answer the following questions as best you can. You have access to the following tools:\n\nSearch: useful for when you need to answer 

In [5]:
cr

ChainRun(id='27323fde-5701-4ed1-b2e9-01f8f55de25e', start_time=datetime.datetime(2022, 12, 23, 22, 0, 36, 461295, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600), 'PST')), end_time=datetime.datetime(2022, 12, 23, 22, 0, 43, 924840, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600), 'PST')), extra={}, error=None, execution_order=1, serialized={'name': 'AgentExecutor'}, inputs={'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"}, outputs={'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."}, child_runs=[{'id': 'cffe3cc7-9a96-408c-8a22-e9837c47dd11', 'start_time': 1671861636.469308, 'end_time': 1671861638.553195, 'extra': {}, 'error': None, 'execution_order': 2, 'serialized': {'name': 'LLMChain'}, 'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?", 'agent_scratchpad

In [8]:
type(cr.child_runs[0])

dict